<a href="https://colab.research.google.com/github/crescendonow/thai_geoparsing/blob/main/1_4BERT_cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Utilities lib for use
import requests
import numpy as np
import pickle
import tqdm
import requests
import torch
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification,\
TrainingArguments, Trainer, DataCollatorForTokenClassification

In [ ]:
#load train, validate and test token by attacut and BERT format
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\train_bert.data', 'rb') as token1:
    train_data = pickle.load(token1)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\eval_bert.data', 'rb') as token2:
    eval_data = pickle.load(token2)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\1.Tokennization\datasets_test.data', 'rb') as token3:
    test_data = pickle.load(token3)    

In [ ]:
#load dictionary index and processing data label_list
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\5.dict\ner_ix.dict', 'rb') as dict_ner:
    ner_ix = pickle.load(dict_ner)
with open(r'C:\Users\sveng\geoparsing_proj\TEXT DATA\TrainingData\6.Processing_variables\ner_list.data', 'rb') as list_ner:
    label_list = pickle.load(list_ner)

In [ ]:
#prepared index to BOI tags, convert index to ner
ix_to_ner = dict((v,k) for k,v in ner_ix.items()) 

In [ ]:
#list of model for use
task = "ner"
public_models = ['xlm-roberta-base', 'bert-base-multilingual-cased'] 
user_name = 'airesearch'
model_name = "wangchanberta-base-att-spm-uncased" 
batch_size = 16

In [ ]:
#define tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{user_name}/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

In [ ]:
#create function for tokenize and reformat
def tokenize_and_align_labels(data):
    tokenized_datasets = []
    for rec in data:
        tokenized_input = tokenizer(rec['tokens'], is_split_into_words=True)
        aligned_labels = [-100 if i is None else rec[f'{task}_tags'][i] for i in tokenized_input.word_ids()]
        rec['input_ids'] = tokenized_input['input_ids']
        rec['attention_mask'] = tokenized_input['attention_mask']
        rec['labels'] = aligned_labels
        tokenized_datasets.append(rec)
    return tokenized_datasets

In [ ]:
#define tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'{user_name}/{model_name}' if model_name not in public_models else f'{model_name}',
                revision='main',
                model_max_length=416,)

In [ ]:
train_bert = tokenize_and_align_labels(train_data)
eval_bert = tokenize_and_align_labels(eval_data)
test_bert = tokenize_and_align_labels(test_data)

In [ ]:
#prepare fine tuning model
model = AutoModelForTokenClassification.from_pretrained(f'{user_name}/{model_name}', num_labels=len(label_list))

#load model from state 20 epoch
path_chkpoint = r'C:\Users\sveng\geoparsing_proj\model_bert\Model1-v120\checkpoint-22000\pytorch_model.bin'
model.load_state_dict(torch.load(path_chkpoint))

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

<All keys matched successfully>

In [ ]:
#training arguments (parameters)
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy="epoch",

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=5,

    seed=9,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine_with_restarts",

)

In [ ]:
#function for convert dataset to batch tensors
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
#computation metrics , we use seqeval for sequence tagging task
metric = load_metric("seqeval")

In [ ]:
#create function for computation metric
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
#fine tuning part by function Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_bert,
    eval_dataset=eval_bert,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
%%time
#call method train
trainer.train()

c:\Users\sveng\anaconda3\envs\geo_bert\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 17955
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 2805


  0%|          | 0/2805 [00:00<?, ?it/s]

The following columns in the training set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to test-ner\checkpoint-500
Configuration saved in test-ner\checkpoint-500\config.json


{'loss': 0.007, 'learning_rate': 1.911713602127308e-05, 'epoch': 0.89}


Model weights saved in test-ner\checkpoint-500\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-500\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


...

Model weights saved in test-ner\checkpoint-2500\pytorch_model.bin
tokenizer config file saved in test-ner\checkpoint-2500\tokenizer_config.json
Special tokens file saved in test-ner\checkpoint-2500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4489
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/281 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.13019289076328278, 'eval_precision': 0.625, 'eval_recall': 0.6932897862232779, 'eval_f1': 0.6573761261261261, 'eval_accuracy': 0.9800254452926208, 'eval_runtime': 247.1631, 'eval_samples_per_second': 18.162, 'eval_steps_per_second': 1.137, 'epoch': 5.0}
{'train_runtime': 15597.085, 'train_samples_per_second': 5.756, 'train_steps_per_second': 0.18, 'train_loss': 0.006073432172683471, 'epoch': 5.0}
CPU times: total: 1d 10h 33min 40s
Wall time: 4h 19min 57s


TrainOutput(global_step=2805, training_loss=0.006073432172683471, metrics={'train_runtime': 15597.085, 'train_samples_per_second': 5.756, 'train_steps_per_second': 0.18, 'train_loss': 0.006073432172683471, 'epoch': 5.0})

In [ ]:
predictions, labels, _ = trainer.predict(test_bert)
predictions = np.argmax(predictions, axis=2)

***** Running Prediction *****
  Num examples = 22444
  Batch size = 16
The following columns in the test set don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: ner_tags, pos_tags, tokens, id. If ner_tags, pos_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.


  0%|          | 0/1403 [00:00<?, ?it/s]

In [ ]:
true_predictions = [
    [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
results = metric.compute(predictions=true_predictions, references=true_labels)
#print(results)

In [ ]:
import pandas as pd
#reformat classification report
def show_3digits(evalue):
    val = round(evalue, 4)
    return val

def eliminate_decimal(evalue):
    val = int(evalue)
    return val

classification_report = pd.DataFrame.from_dict(results).transpose()

classification_report['precision'] = classification_report['precision'].apply(show_3digits)
classification_report['recall'] = classification_report['recall'].apply(show_3digits)
classification_report['f1'] = classification_report['f1'].apply(show_3digits)
classification_report['number'] = classification_report['number'].apply(eliminate_decimal)

In [ ]:
print(classification_report)

                   precision  recall      f1  number
ACP                   0.9278  0.9184  0.9231     784
ADMIN                 0.9199  0.9391  0.9294    4332
BSN                   0.8250  0.9141  0.8673     454
DEP                   0.9315  0.9463  0.9389    1882
FPLACE                0.8822  0.9224  0.9018    1185
GOV                   0.9038  0.8782  0.8908     353
HP                    0.8685  0.9307  0.8986     433
MKT                   0.9479  0.9446  0.9463     289
MON                   0.8158  0.7209  0.7654      43
NAT                   0.8303  0.8916  0.8599     203
OTHER                 0.9655  0.8750  0.9180      32
RCT                   0.9100  0.9529  0.9309     785
RES                   0.8596  0.9111  0.8846     551
ROAD                  0.9068  0.9265  0.9166    2490
RP                    0.9560  0.9702  0.9631    1075
RT                    0.8861  0.9288  0.9070    2220
STORE                 0.8247  0.8292  0.8269     363
TRAN                  0.8784  0.9144  0.8961  

In [ ]:
#line notify to me if complete
token_jobnotify = 'M1qygx1uFYuGtgo6qoKPboB4sAKmAhGuwXdL7q54TvP'
msg = f""" run process เสร็จละนะจ๊ะที่รัก เข้ามาดูเค้าหน่อยน้า 
           f1 = {classification_report.loc['overall_f1']['f1']} """
url = 'https://notify-api.line.me/api/notify'
headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+token_jobnotify}
r = requests.post(url, headers=headers, data = {'message':msg})
print (r.text)

{"status":200,"message":"ok"}


In [ ]:
#Save model by this path
folder_name = 'Model1-v4'
save_path = r"C:\Users\sveng\geoparsing_proj\model_bert"
fullpath = save_path+'\\'+folder_name

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(fullpath)

# save last weight pretrained
model.save_pretrained(fullpath)

Configuration saved in C:\Users\sveng\geoparsing_proj\model_bert\Model1-v4\config.json
Model weights saved in C:\Users\sveng\geoparsing_proj\model_bert\Model1-v4\pytorch_model.bin
